# Tech Forum 2025 (severe) imbalanced data


## Supervised machine learning "binary classification" 
I will speak about supervised machine learning "binary classification" 

in the context of fraud detection on the card credit transactions

# My own libraries, variables, data sets ...

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import mccmanagement 
import result

reload(fd)
reload(sd)
reload(mccmanagement)
reload(result)

print('done')

In [ ]:
# standard librairies
import numpy as np
import pandas as pd
import numpy
import pandas
import sklearn
import imblearn
import dataframe_image as dfi
import dataimport as di
from datetime import datetime

#diagrams
import seaborn as sns
import matplotlib.pyplot as plt

#Classifiers
from sklearn.dummy import DummyClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#metrics
from sklearn.metrics import accuracy_score



In [ ]:
source='WL'
date='20241118'

In [ ]:
def IR(dfTrx, target='Class'):
    majorityCount=dfTrx[target].value_counts()[0]
    minorityCount=dfTrx[target].value_counts()[1]
    return majorityCount/minorityCount

def fraudRate(dfTrx, target='Class'):
    majorityCount=dfTrx[target].value_counts()[0]
    minorityCount=dfTrx[target].value_counts()[1]
    return minorityCount/(minorityCount+majorityCount)

def printMessage(message,figure,format="{0}"):
    fullMessage=message +" \033[1m\033[94m"+format+"\033[0m"
    print(fullMessage.format(figure))
    

In [ ]:

dfTrxRaw = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrxRaw.head(5)

dfTrxClean22 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
predictors = fd.getPredictors(dfTrxClean22)
dfTrxScaled=fd.getStandardScaledData('export'+date+'.csv',source,predictors)
dfTrxScaled.head(5)

In [ ]:
row = 10000
cols = 5

Random = np.random.randint(low=0, high=100, size=(row, cols))

dfCreditApproval = pd.DataFrame(Random)

#print(df)

dfCreditApproval['Class'] = np.random.choice([0, 1], p=[0.75, 0.25], size=len(dfCreditApproval))
dfCreditApproval.Class.value_counts()

In [ ]:
row = 10000
cols = 5

Random = np.random.randint(low=0, high=100, size=(row, cols))

dfBankChurn = pd.DataFrame(Random)

#print(df)

dfBankChurn['Class'] = np.random.choice([0, 1], p=[0.8, 0.2], size=len(dfCreditApproval))
dfBankChurn.Class.value_counts()

# IR Imbalance Ratio

In [ ]:
#The value IR imbalance ratio is defined for the binary classification by the ratio number of negative class examples on the number of positive class examples. It should be greater or equal to 1.
#IR = (number positive examples) / (number of negative examples)
#IR >=1
#Because the fraud rate is very low we can set that
#IR =1/(Fraud rate at transaction level)


#-	Credit approval (refused 75%, approved 25%)
printMessage("IR for credit approval",IR(dfCreditApproval),"{0:.2f}")

#-	Bank churn (retained 80%, churn 20%) 
printMessage("IR for bank churn",IR(dfBankChurn),"{0:.2f}")


In [ ]:

printMessage("IR for fraud card credit transactions",IR(dfTrxRaw),"{0:.2f}")
printMessage("Fraud rate credit transactions",fraudRate(dfTrxRaw,'Class'),"{0:.5f}")



## Count plot

In [ ]:
fig, ax =plt.subplots(1,2)
sns.countplot(x='Class', data=dfCreditApproval, ax=ax[0], orient="v") 
ax[0].set_title("Credit approval")
sns.countplot(x='Class', data=dfTrxRaw, ax=ax[1], orient="v") 
ax[0].set_title("Credit card fraud")
fig.show()

## Pie diagram
Yes pie diagram is sometimes misleading

In [ ]:
# add pie diagram
fig, ax =plt.subplots(1,2)
fig.set_figheight(6)
fig.set_figwidth(12)
ax[0].pie(dfCreditApproval['Class'].value_counts(),autopct='%1.1f%%', labels=['Refused','Approved'], colors=['yellowgreen','r'])
ax[0].set_title("Credit approval")
ax[1].pie(dfTrxRaw['Class'].value_counts(),autopct='%1.1f%%', labels=['Genuine','Fraud'], colors=['yellowgreen','r'])
ax[1].set_title("Credit card fraud")
fig.show()

# Libraries used 

In [ ]:
print("Numpy",numpy.__version__)
print("Pandas",pandas.__version__)
print("Sklearn",sklearn.__version__)
print("Imblearn",imblearn.__version__)
print("Imbalanced-learn is fully compliant scikit-learn")

# Choose the good metrics

In [ ]:
# issue with accuracy
# Accuracy =(Num Correct Predictions) / ( Num Total Predictions)

In [ ]:
x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrxRaw,[])

modelClf = DummyClassifier(strategy="most_frequent")
modelClf.fit(x_train, y_train)
y_predictTest = modelClf.predict(x_test)
printMessage("","Dummy classifier")
printMessage("Accuracy score",accuracy_score(y_test,y_predictTest),"{0:.4f}")


In [ ]:
printMessage("Accuracy =","(Num Correct Predictions) / ( Num Total Predictions)")
fd.show_confusion_matrix(y_test, y_predictTest)


![stop.png](../imgs/presentation/stop.png)
v>
## Accuracy is not the good metrics for imbalanced data
## You can use
* F1 Score (calculated from precision and recall
    * Precision = TP / (TP + FP)    * 
Recall = TP / (TP + FN)
* there are many other metg)cs

* F1 Score (calculated from precision and recall
    * Precision = TP / (TP + FP)
    * Recall = TP / (TP + FN)

![confusionMatrix.png](../imgs/presentation/confusionMatrix.png)

# IV (information value)

## Definition IV
information value (IV) is a metric used to assess the predictive power of a feature
in capturing the relationship between an independent variable and a binary dependent variable. 
It helps determine which features are most important for predicting a binary outcome.

This is not present in scitkit-learn


In [ ]:
data = result.load_features_IV_result()

data = data[ 
    ((data.IV>0.0) & (data.IV <0.03))
    | (data.IV> 1.7)
    ]
style_test=pd.DataFrame(data=data,
             columns=['Feature', 'IV']).style.background_gradient(axis='index')
style_test.hide(axis="index")

style_test

## Good example Visa score

In [ ]:
sns.violinplot(x='Class', y='vaa_score', data=dfTrxRaw, palette=['#2f7ed8', '#d6604d'])
plt.xlabel('Class (0 = Genuine, 1 = Fraud)')
plt.ylabel('mcd_fraud_score')

plt.tight_layout()
plt.suptitle("Visa score per class", y=1.02, fontsize=16)
plt.show()

## Bad example TRX_3D_SECURED

In [ ]:
printMessage("IV for TRX_3D_SECURED",fd.calc_iv(dfTrxRaw,'TRX_3D_SECURED','Class',pr=0),"{0:.3f}")

print("TRX_3D_SECURED values")
print(dfTrxRaw.TRX_3D_SECURED.value_counts())

print("")
print("------------------------")
print("Fraud rate 'TRX_3D_SECURED'=1")
temp=dfTrxRaw[(dfTrxRaw['TRX_3D_SECURED']==1)]
print(temp.Class.value_counts(normalize=True))

print("")
print("------------------------")
print("Fraud rate 'TRX_3D_SECURED'=0")
temp=dfTrxRaw[(dfTrxRaw['TRX_3D_SECURED']==0)]
print(temp.Class.value_counts(normalize=True))


In [ ]:
fig, ax =plt.subplots(1,3)
fig.set_figheight(8)
fig.set_figwidth(24)
ax[0].pie(dfTrxRaw['TRX_3D_SECURED'].value_counts(),autopct='%1.1f%%', labels=['0','1'], colors=['yellowgreen','r'],
         textprops={'fontsize': 25})
ax[0].set_title("TRX_3D_SECURED value", fontsize=25)

temp1=dfTrxRaw[(dfTrxRaw['TRX_3D_SECURED']==1)]
ax[1].pie(temp1['Class'].value_counts(),autopct='%1.2f%%', labels=['0','1'], colors=['yellowgreen','r'],
         textprops={'fontsize': 25})
ax[1].set_title("Fraud rate for TRX_3D_SECURED=1", fontsize=25)

temp2=dfTrxRaw[(dfTrxRaw['TRX_3D_SECURED']==0)]
ax[2].pie(temp2['Class'].value_counts(),autopct='%1.2f%%', labels=['0','1'], colors=['yellowgreen','r'],
         textprops={'fontsize': 25})
ax[2].set_title("Fraud rate for TRX_3D_SECURED=0", fontsize=25)

fig.show()


# Stratified train-test split

In [ ]:
## Important
printMessage("During the split train test data ","use option stratify= target class...")

In [ ]:
from sklearn.model_selection import train_test_split
TEST_SIZE=0.2
RANDOM_STATE=44

x_train0, x_test0, y_train0, y_test0 = train_test_split(dfTrxRaw, dfTrxRaw['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrxRaw['Class'],
                                                        random_state = RANDOM_STATE)

x_train1, x_test1, y_train1, y_test1 = train_test_split(dfTrxRaw, dfTrxRaw['Class'], test_size = TEST_SIZE, 
                                                        random_state = RANDOM_STATE)


In [ ]:
fig, ax =plt.subplots(1,3)
fig.set_figheight(8)
fig.set_figwidth(24)
ax[0].pie(dfTrxRaw['Class'].value_counts(),autopct='%1.3f%%', labels=['0','1'], colors=['yellowgreen','r'],
         textprops={'fontsize': 25})
ax[0].set_title("Initial data", fontsize=25)

ax[1].pie(y_test0.value_counts(),autopct='%1.3f%%', labels=['0','1'], colors=['yellowgreen','r'],
         textprops={'fontsize': 25})
ax[1].set_title("Test data stratify", fontsize=25)

ax[2].pie(y_test1.value_counts(),autopct='%1.3f%%', labels=['0','1'], colors=['yellowgreen','r'],
         textprops={'fontsize': 25})
ax[2].set_title("Test data not stratify", fontsize=25)

fig.show()


# Class-weight (scitkit-learn)

## Explanation
The class_weight parameter in Scikit-learn is used to assign weights to the classes in the data. 
This is particularly useful when you have an imbalanced dataset. 
By assigning a higher weight to the minority class and a lower weight to the majority class, 
the model is forced to pay more attention to the minority class during training.

In [ ]:
x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrxScaled,predictors)


max_depthFound=15
min_samples_leafFound=4
min_samples_splitFound=21
criterionFound='entropy'

In [ ]:
modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': criterionFound, 'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 'min_samples_split': min_samples_splitFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

fd.print_scores(y_test, predsTest,'f1')
fd.show_confusion_matrix(y_test, predsTest)


In [ ]:
class_weightFound={0: 1, 1: 5}

modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': criterionFound, 'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 'min_samples_split': min_samples_splitFound,
            'class_weight':class_weightFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

fd.print_scores(y_test, predsTest,'f1')
fd.show_confusion_matrix(y_test, predsTest)

## 'class_weight' parameter does not exist for the boosting (AdaBoost, GradientBoosting)

# Sampling
* oversampling
* undersampling
* combined method

Without sampling, for the fraud detection the result are very poor


In [ ]:
# stop scaling needed (for advanced technics

In [ ]:


data=result.load_performance_nextdays_result()

data['f1']=np.around((data['F1 Day2']+data['F1 Day3']+data['F1 Day4'])/3, decimals=3)

data = data[['Name','ExtraParameters','f1']]
data = data[ 
    (data.Name!='BaggingClassifier')
    & (data.Name!='BalancedRandomForestClassifier')
    & (data.Name!='EasyEnsembleClassifier')
    & (data.Name!='CatBoostClassifier')
    & (data.Name!='XGBClassifier')
    & (data.Name!='3-ExtraTreesClassifier')
    #& (data.Name!='GradientBoostingClassifier')
    ]
res = pd.crosstab(data['ExtraParameters'], data['Name'], values=data['f1'], aggfunc='mean')
style_test=res.style.background_gradient(axis='index')

styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "200%"),
                              ("color", 'black')])]

style_test.set_caption("F1 result").set_table_styles(styles)
dfi.export(style_test, '../imgs/temp.png',table_conversion = 'matplotlib')

style_test

In [ ]:
#Stop Scaling needed

# Choose the good algorithm
* ensemble bagging
* ensemble boosting

# Prediction diagram

## Classification or Scoring
in scikit-learn classifier there is also a score (between 0.0 and 1.0)
if the score is greater than a threshold (0.5) the record is classfied on one side or one the other side.
let us create an histogram with these scores.


In [ ]:

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrxScaled,predictors)

modelClf = BalancedRandomForestClassifier(random_state=42)
n_estimatorsFound=80
max_depthFound=12
parameters={'max_depth': max_depthFound, 'n_estimators':n_estimatorsFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

#f1Train=fd.print_scores(y_train, predsTrain,'f1', False)
f1Test=fd.print_scores(y_test, predsTest,'f1')


In [ ]:
fd.show_prediction_graphNoCorrection(modelClf, x_test,y_test,None)

In [ ]:
fd.show_prediction_graph(modelClf, x_test,y_test,None)

# Conclusion
* There are tools to manage imbalanced data
* But for fraud detection the imbalance is severe and there are some limitations